In [1]:
### Load the keys from the environment variable in .env file
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:

from langchain_groq import ChatGroq
# model is an instance of ChatGroq. This is your Large Language Model (LLM) component. 
# When model is "invoked" (either directly or as part of a chain), it takes an input 
# (like your message list) and generates a response, which is typically 
# a BaseMessage object (or a list of them).
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000015CC2971010>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000015CC2971BE0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

message=[
    SystemMessage(content="Translate this message from English to French"),
    HumanMessage(content="Hello how are you?")
]

result = model.invoke(message)

In [5]:
from langchain_core.output_parsers import StrOutputParser
# parser: parser is an instance of StrOutputParser. This is an output parser component.
# Its job is to take the raw output from the LLM 
# (which might be a complex object containing message type, content, etc.)
# and transform it into a more usable format, in this case, a simple string.
parser = StrOutputParser()
parser.invoke(result)

'Bonjour, comment vas-tu ? \n\n(This is the most common and informal way to say it)\n\n\nYou can also say:\n\n* **Bonjour, comment allez-vous ?** (More formal)\n* **Salut, comment ça va ?** (Very informal)\n'

In [6]:
'''
LangChain Expression Language (LCEL) is a syntax within the LangChain framework that allows 
developers to easily define and combine different components into complex workflows, particularly 
for building applications powered by large language models (LLMs).
'''


# Chain of components
# the line chain = model | parser is a key part of the LangChain Expression Language (LCEL) and 
# represents a chaining mechanism that connects different components together.
# | (Pipe Operator): In LCEL, the pipe operator | is used to "pipe" the output of the component 
# on its left to the input of the component on its right. It signifies a sequential execution flow.
chain=model|parser
chain.invoke(message)

# In this method instead of invoking model.invoke(message) getting output in result
# and then passing the result to parser.invoke(result)
#  we are using chaining to get the desired result and pipe passes the output

"Bonjour, comment allez-vous\xa0? \n\n\nYou can also use:\n\n* **Salut, ça va\xa0?** (Informal)\n* **Comment vas-tu\xa0?** (Informal, singular) \n\n\nLet me know if you have any other phrases you'd like translated!\n"

In [7]:
### Prompt Templates
# ChatPromptTemplate: This class is specifically designed to create prompts for chat models 
# (like the ChatGroq model we are using). Chat models typically take a list of messages,
#  each with a role (e.g., "system", "human", "ai"), rather than a single string. 
# ChatPromptTemplate helps you construct these lists of messages in a structured and reusable way.
from langchain_core.prompts import ChatPromptTemplate

# This defines a simple string variable generic_template.
# {language} is a placeholder within the string. It signifies that this part of the template will be 
# filled in dynamically when the prompt is actually used.

generic_template ="Translate Message into the {language}:"

# It creates an instance of ChatPromptTemplate. Let's look at the list passed to its constructor: 
# [("system",generic_template),("user"),"{text}"]. This list defines the structure of your chat prompt, 
# which will eventually be sent to the LLM.

prompt = ChatPromptTemplate(
    [("system",generic_template),("user"),"{text}"]
)

#  Below line creates a structured prompt, does not involve the model yet
result = prompt.invoke({"language":"french","text":"hello"})
# this lines only defines the sequence of chain, but does not execute it
chain = prompt|model|parser
#  This is where invocation is happening, notice, I cannot use the result and have to pass
#  parameters again because only runnable type can be used in a chain.
chain.invoke({"language":"french","text":"hello"})


'Bonjour ! 😊  \n'